In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
)

template = PromptTemplate.from_template("What is the capital of {country}?")

prompt = template.format(country="France")

chat.predict(prompt)

In [ ]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}"),
    ("ai", "Hello, my name is {name}"),
    ("human", "what is the capital of {country}? And what is your name?")
])

prompt = template.format_messages(language="korean", name="Jamtol", country="Korea")

chat.predict_messages(prompt)

In [ ]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    

p = CommaOutputParser()

p.parse("Hello, my name, is Jamtol")

In [30]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items}. Do NOT reply with anything else."),
    ("human", "{question}")
])

In [ ]:
chain = template | chat | CommaOutputParser()
chain.invoke({"question": "what are the pokemons?", "max_items": 5})

In [33]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

chef_template = ChatPromptTemplate.from_messages([
    ("system", "You are a chef. You create a recipe for a dish."),
    ("human", "{dish}")
])

chef_chain = chef_template | chat

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef. You find alternative ingredients for a dish."),
    ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({"dish": "Spaghetti Carbonara"})

### 4.1 FewShotPromptTemplate

In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts import PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ],
)

In [ ]:
example = [
    {
        "question": "What do you know about Greece?",
        "answer": """
I know this:
Capital: Athens
Language: Greek
Currency: Euro
Food: Moussaka
"""
    },
    {
        "question": "What do you know about France?",
        "answer": """
I know this:
Capital: Paris
Language: French
Currency: Euro
Food: Coq au Vin
"""
    }
]

example_template = """
Question: {question}
Answer: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

prompt = FewShotPromptTemplate(
    examples=example,
    example_prompt=example_prompt,
    suffix="Question: What do you know about {Country}?",
    input_variables=["Country"],
)

chain = prompt | chat

chain.invoke({"Country": "Germany"})

### 4.2 FewShotChatMessagePromptTemplate

In [16]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ],
)

example = [
    {
        "question": "Greece",
        "answer": """
I know this:
Capital: Athens
Language: Greek
Currency: Euro
Food: Moussaka
"""
    },
    {
        "question": "France",
        "answer": """
I know this:
Capital: Paris
Language: French
Currency: Euro
Food: Coq au Vin
"""
    }
]

example_template = """
Question: {question}
Answer: {answer}
"""

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {question}?"),
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    examples=example,
    example_prompt=example_prompt,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that can answer questions about countries."),
    example_prompt,
    ("human", "What do you know about {question}?"),
])

chain = final_prompt | chat

chain.invoke({"question": "Thailand"})

I know this:
Capital: Bangkok
Language: Thai
Currency: Thai Baht
Food: Pad Thai

AIMessageChunk(content='I know this:\nCapital: Bangkok\nLanguage: Thai\nCurrency: Thai Baht\nFood: Pad Thai')

### 4.3 LengthBasedExampleSelector

In [28]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ],
)

examples = [
    {
        "question": "What do you know about Greece?",
        "answer": """
I know this:
Capital: Athens
Language: Greek
Currency: Euro
Food: Moussaka
"""
    },
    {
        "question": "What do you know about France?",
        "answer": """
I know this:
Capital: Paris
Language: French
Currency: Euro
Food: Coq au Vin
"""
    },
    {
        "question": "What do you know about Korea?",
        "answer": """
I know this:
Capital: Seoul
Language: Portuguese
Currency: Real
Food: Feijoada
"""
    }
]

class RandomExampleSelector(BaseExampleSelector):
    """Select random examples to use based on the input variables."""
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)
    
    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]

example_template = """
Question: {question}
Answer: {answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

example_selector = RandomExampleSelector(
    examples = examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Question: What do you know about {Country}?",
    input_variables=["Country"],
)

prompt.format(Country="Brazil")

'\nQuestion: What do you know about Korea?\nAnswer: \nI know this:\nCapital: Seoul\nLanguage: Portuguese\nCurrency: Real\nFood: Feijoada\n\n\n\nQuestion: What do you know about Brazil?'

### 4.4 Serialization and Composition

In [42]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
# from langchain.prompts import load_prompt

# prompt = load_prompt("./prompt.json")
# prompt = load_prompt("./prompt.yaml")

from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler()
    ],
)


intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
    """
)

example = PromptTemplate.from_template(
    """
    This is a example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]

full_prompt = PipelinePromptTemplate(
    final_prompt = final,
    pipeline_prompts = prompts,
)

chain = full_prompt | chat

chain.invoke({
    "character": "a pirate", 
    "example_question": "What is your location?", 
    "example_answer": "Arrrr! That is a secret! Arrrr!", 
    "question": "What is your favorite food?"
})

Arrr! Me favorite food be a good ol' plate of salted fish and hardtack! Arrr!

AIMessageChunk(content="Arrr! Me favorite food be a good ol' plate of salted fish and hardtack! Arrr!")

### 4.5 Caching

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(SQLiteCache("cache.db"))
set_debug(True)

chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

chat.invoke("How do you make italian pasta")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere is a step-by-step guide on how to make Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, divide the dough into smaller portions and roll each portion out into a thi

AIMessage(content='To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere is a step-by-step guide on how to make Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, divide the dough into smaller portions and roll each portion out into a thin sheet using a pasta machine or a rolling pin.\n7. Cut the dough into your desired shape, such as fettuccine, spaghetti, or ravioli.\n8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes or until al 

In [2]:
chat.invoke("How do you make italian pasta")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pasta"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere is a step-by-step guide on how to make Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, divide the dough into smaller portions and roll each portion out into a thi

AIMessage(content='To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere is a step-by-step guide on how to make Italian pasta:\n\n1. On a clean work surface, pour the flour and make a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, gradually mix the eggs into the flour until a dough forms.\n4. Knead the dough for about 10 minutes until it becomes smooth and elastic. If the dough is too dry, add a little water. If it is too wet, add a little more flour.\n5. Wrap the dough in plastic wrap and let it rest for at least 30 minutes.\n6. After resting, divide the dough into smaller portions and roll each portion out into a thin sheet using a pasta machine or a rolling pin.\n7. Cut the dough into your desired shape, such as fettuccine, spaghetti, or ravioli.\n8. Cook the pasta in a large pot of boiling salted water for 2-3 minutes or until al 

### 4.6 Serialization

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1,
)

with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for soju")
    b = chat.predict("What is the recipe for bread")
    print(a, b, "\n")
    print(usage)
    print(usage.total_tokens)
    print(usage.prompt_tokens)
    print(usage.completion_tokens)
    print(usage.total_cost)

Ingredients:
- 1 cup of rice
- 1 cup of water
- 1 tablespoon of nuruk (fermentation starter)
- 1 tablespoon of sugar

Instructions:
1. Rinse the rice thoroughly and soak it in water for at least 1 hour.
2. Drain the rice and place it in a steamer basket. Steam the rice for about 30 minutes, or until it is cooked through.
3. Transfer the cooked rice to a large bowl and mash it with a wooden spoon or a potato masher.
4. In a separate bowl, mix the nuruk and sugar with water until dissolved.
5. Add the nuruk mixture to the mashed rice and mix well.
6. Cover the bowl with a clean cloth and let it ferment in a warm place for about 3-4 days, stirring occasionally.
7. After the fermentation process is complete, strain the mixture through a cheesecloth to remove any solids.
8. Transfer the liquid to a clean bottle and store it in the refrigerator for at least 1 week before serving.

Enjoy your homemade soju! Ingredients:
- 3 1/2 cups all-purpose flour
- 1 packet active dry yeast
- 1 1/2 cups w

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI
from langchain.llms.loading import load_llm

# chat = OpenAI(
#     temperature=0.1,
#     max_tokens=456,
#     model = "gpt-3.5-turbo",
# )

# chat.save("model.json")

chat = load_llm("model.json")

chat

c:\Users\Jamtol\anaconda3\envs\full-gpt\Lib\site-packages\langchain\llms\openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
c:\Users\Jamtol\anaconda3\envs\full-gpt\Lib\site-packages\langchain\llms\openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


OpenAIChat(client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_kwargs={'temperature': 0.1, 'max_tokens': 456, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}})